In [ ]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

In [ ]:
################
# Start FSLeyes (very neat tool to visualize MRI data of all sorts) within Python
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

In [ ]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def direct_file_download_open_neuro(file_list, file_types, dataset_id, dataset_version, save_dirs):
    # https://openneuro.org/crn/datasets/ds004226/snapshots/1.0.0/files/sub-001:sub-001_scans.tsv
    for i, n in enumerate(file_list):
        subject = n.split('_')[0]
        download_link = 'https://openneuro.org/crn/datasets/{}/snapshots/{}/files/{}:{}:{}'.format(dataset_id, dataset_version, subject, file_types[i],n)
        print('Attempting download from ', download_link)
        download_url(download_link, op.join(save_dirs[i], n))
        print('Ok')
        
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

In [ ]:
dataset_id = 'ds000171' # This is the dataset ID for the OpenNeuro dataset we are using.
subject_id = 'control01'    # This is the subject ID that we are using.

sample_path = "/home/jovyan/Data/dataset" 
mkdir_no_exist(sample_path) # Creates the directory if it does not already exist.

bids_root = op.join(sample_path, dataset_id) # Defines the BIDS root directory based on the dataset ID.
deriv_root = op.join(bids_root, 'derivatives') # Defines the path for the derivatives directory, following BIDS standards.
preproc_root = op.join(bids_root, 'derivatives', 'preprocessed_data') # Defines the path for the preprocessed data.

mkdir_no_exist(bids_root) # Creates the BIDS root directory if it doesn't exist.

subject_dir = 'sub-{}'.format(subject_id) # Formats the subject ID following the BIDS convention.

In [ ]:
###################
# OpenNeuro download.
###################
subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, # OpenNeuro assigns a unique ID for each dataset.
                "--target-dir", bids_root,  # Specifies where the data will be saved locally.
                "--include", op.join(subject_dir, '*'), # Downloads all files within the subject_dir   
               ], check=True) # Ensures that the process raises an error if the download fails.

###################
# Create folders relevant for preprocessing.
# In BIDS, ANY changes or preprocessing must be stored in the derivatives folder to keep the original data unaltered.
###################
mkdir_no_exist(op.join(bids_root, 'derivatives')) # Ensures the derivatives folder exists within the BIDS root.

preproc_root = op.join(bids_root, 'derivatives', 'preprocessed_data') # Defines the directory for storing preprocessed data.
mkdir_no_exist(preproc_root) # Creates the preprocessed data directory if it doesn’t exist.
mkdir_no_exist(op.join(preproc_root, 'sub-{}'.format(subject_id))) # Creates a folder for the subject in the preprocessed data directory.
mkdir_no_exist(op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat')) # Creates the folder for anatomical data.
mkdir_no_exist(op.join(preproc_root, 'sub-{}'.format(subject_id), 'func')) # Creates the folder for functional (fMRI) data.
mkdir_no_exist(op.join(preproc_root, 'sub-{}'.format(subject_id), 'fmap')) # Creates the folder for field map data.


In [ ]:
print_dir_tree(bids_root, max_depth=4)

In [ ]:
# Define the path to the functional JSON file in the BIDS dataset.
# 'bids_root' is the root directory for the BIDS dataset.
# 'sub-control01' is the subject's folder name, following BIDS conventions.
# 'func' is the folder where functional data (fMRI) is stored.
# 'sub-control01_task-music_run-1_bold.json' is the specific JSON file containing metadata for the fMRI scan.
json_path = op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_task-music_run-1_bold.json') 

# Use the function 'get_json_from_file' to read the JSON file and return its contents as a dictionary.
data = get_json_from_file(json_path)

# Display the contents of the JSON file stored in 'data'
data


In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz')) 

# Skull stripping

###  Preprocessing and BIDs
An important part of **anatomical** preprocessing is to remove the skull around the brain.
To adhere to the BIDs format, all modified files should be put in a new folder, called derivatives, such that you always have clean data in the source directory. The derivatives folder can be used for different preprocessing and treatments, each needing their own subfolders. In our case, we've created a single folder, preprocessed_data, hence the following structure:

In [ ]:
def get_skull_stripped_anatomical(bids_root, preproc_root, subject_id, robust=False): 
    """
    Function to perform skull-stripping (removing the skull around the brain).
    This is a simple wrapper around the brain extraction tool (BET) in FSL's suite.
    It assumes data to be in the BIDS format.

    The method also saves the brain mask which was used to extract the brain.

    The brain extraction is conducted only on the T1w of the participant.

    Parameters
    ----------
    bids_root: string
        The root of the BIDS directory.
    preproc_root: string
        The root of the preprocessed data, where the result of the brain extraction will be saved.
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """

    # Format the subject ID following the BIDS convention (e.g., 'sub-001')
    subject_dir = 'sub-{}'.format(subject_id) 
    
    # Define the path to the anatomical T1-weighted image for the subject
    anatomical_path = op.join(bids_root, subject_dir, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id)) 

    # Define the path where the skull-stripped brain will be saved
    betted_brain_path = op.join(preproc_root, subject_dir, 'anat', 'sub-{}_T1w'.format(subject_id)) 

    # Run FSL's brain extraction tool (BET) using the os.system command
    # - If 'robust' is True, the '-R' option is used for robust center estimation
    # - Otherwise, no additional flag is used
    os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R' if robust else ''))

    # Print a message indicating the process is complete
    print("Done with BET.")

#  This sets the path where the brain mask will be stored. Takes as input the file stored in betted_brain_path from BET
resulting_mask_path = op.join(preproc_root, subject_dir, 'anat', 'sub-{}_T1w_mask'.format(subject_id) ) 

get_skull_stripped_anatomical(bids_root, preproc_root, "control01") 

In [ ]:
fsleyesDisplay.load(resulting_mask_path)

### Improving the fit
If you look a bit into bet's documentation, you'll quickly find that there are parameters with which you can play; robust brain centre estimation and fractional intensity threshold. To demonstrate the importance and impact of these parameters, let's use a robust brain center estimation.

In [ ]:
get_skull_stripped_anatomical(bids_root, preproc_root, "control01", robust=True)

In [ ]:
# Reset any existing overlays in FSLeyes to start with a clean display
fsleyesDisplay.resetOverlays()

# Load the T1-weighted anatomical image of subject 'sub-001' into FSLeyes
# The path is constructed by joining the BIDS root with the subject directory, the 'anat' folder,
# and the file name 'sub-001_T1w.nii.gz' which follows the BIDS convention.
fsleyesDisplay.load(op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz'))

# Load a mask overlay onto the anatomical image
# The 'resulting_mask_path' variable should point to a mask file that highlights specific regions
# or structures of interest within the anatomical image.
fsleyesDisplay.load(resulting_mask_path)


### 1.1.4 Manual corrections
If you really want good fit, you might want to resort to **manually correcting the mask**. Run the code above to check whether the result is satisfactory or not. 

FSLeyes readily allows you to do such things! While on FSLeyes, press **Alt + E** to open the editing interface.

>Edit


You can then escape back to the non-edit mode by pressing again **Alt + E**


In [ ]:
##################################
# Fill me with the code to use your mask
# To help you, we provide you with the skeleton of two potential approaches.
# You can fill either of them. Do not forget to test them by visualizing the result!
# For fslmaths, you can either read the documentation, or execute it without argument by running os.system('fslmaths')
##################################


# Based on the code and the explanations I provided earlier, the value to insert in `mask_path` should be the path to the mask file generated during the skull-stripping process using BET. 
# This file is indicated by `resulting_mask_path`.

# Option 1: Pythonic approach.
def apply_python_mask_approach(img_path, mask_path, masked_img_path):
    """
    The first approach, Pythonic way. The goal is, given a mask, to apply it to a T1 image where the brain is to be extracted.

    Parameters
    ----------
    img_path: str
        Path to the image on which we would like to apply the mask (in this case, the T1 with the skull still on). Should be a .nii.gz file.
    mask_path: str
        Path to the mask you would like to apply to your image. Should be a .nii.gz file, containing only binary values (0 or 1).
    masked_img_path: str
        Path to which the resulting masked image will be saved. 
    """
    import nibabel as nib

    # Load both the T1 image and the mask from disk
    img = nib.load(img_path)
    mask = nib.load(mask_path)
    
    # Extract the image data and mask data as numpy arrays. 
    img_data = img.get_fdata()
    mask_data = mask.get_fdata()

    #######################
    # Solution 1
    # Create an empty image array and fill it with values where the mask is greater than 0
    #######################
    # Initialize an empty array with the same shape as the T1 image
    saved_img_data = np.zeros(img_data.shape)
    # Fill the array with image data only where the mask is greater than 0 (brain region)
    saved_img_data[mask_data > 0] = img_data[mask_data > 0]

    # Save the masked image to disk by creating a new Nifti image and writing it out
    img_out = nib.Nifti1Image(saved_img_data, img.affine, img.header)
    nib.save(img_out, masked_img_path)

    #######################
    # Solution 2
    # An alternative approach: set all values outside the mask (where mask = 0) to zero.
    #######################
    
    # Set all image data values to 0 where the mask is equal to 0 (outside the brain region)
    img_data[mask_data == 0] = 0
    
    # Save the modified image to disk, creating a new Nifti image and writing it out
    img_out = nib.Nifti1Image(img_data, img.affine, img.header)
    nib.save(img_out, masked_img_path)
    
def apply_fsl_math_approach(img_path, mask_path, masked_img_path):
    ###########################
    # Solution
    # Based on fslmaths documentation, the -mas option is used to apply a mask to the image.
    ###########################
    os.system('fslmaths {} -mas {} {}'.format(img_path, mask_path, masked_img_path))
    

# Define the path to the original T1 image of the brain
anatomical_path = op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz') # The original brain

# Define the path where the skull-stripped brain will be saved (in the derivatives folder)
betted_brain_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz') 

# Define the path to the brain mask generated from the skull-stripping process
resulting_mask_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_mask.nii.gz') 

########################
# CHOOSE ONE OF THE TWO TO IMPLEMENT IT AND LAUNCH IT
########################
# Apply the mask using FSL's fslmaths tool. This approach applies the mask directly using command-line tools.
# apply_fsl_math_approach(anatomical_path, resulting_mask_path, betted_brain_path)

# Apply the mask using the Python-based approach. This reads the images into Python, applies the mask, and saves the result.
apply_python_mask_approach(anatomical_path, resulting_mask_path, betted_brain_path)


In [ ]:
# As always, do not forget to visualize what you have done. If you did a proper job, you should now see the brain without any skull!

fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

# Note: If you get a RuntimeError: wrapped C/C++ object of type OrthoEditActionToolBar has been deleted, do not worry, 
# this is simply because you forgot to escape back to non-edit mode (Alt+E) before resetting the overlay, but it is a benign error :)

## 1.2 Tissue segmentation

For the purpose of analysis, it can be useful to separate the tissues into tissue classes; in particular extracting the white matter, grey matter and cerebrospinal fluid (abreviated as CSF) is very interesting in fMRI analysis

It is perfectly possible (even likely) that FSLeyes will stop responding over the course of this lab. This is perfectly normal! Simply wait for whichever function (such as FAST) to finish and it should start responding again, don't worry too quickly, be patient :)</p>


Note that FAST will take one or two minutes to run, this is expected, do not panic :)

In [ ]:
# Define the path to the original T1-weighted anatomical image for subject 'sub-001'.
# This is the raw, unprocessed anatomical scan with the skull.
anatomical_path = op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz')

# Define the path to the skull-stripped image (brain-extracted) for subject 'sub-001'.
# This path points to the image generated after applying BET, stored in the derivatives folder.

# Recall: betted_brain_path = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w.nii.gz') 

bet_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w')

#########################
# Solution
# We need to apply FAST (FSL’s Automated Segmentation Tool) to the brain-extracted image.
# Therefore, we use the BET path as the target since it has the skull removed.
##########################

fast_target = bet_path # Assign the skull-stripped image path (bet_path) to fast_target.
# Note: You could experiment with using anatomical_path instead, but the brain-extracted image (bet_path) is more appropriate.

# Remove any existing FAST segmentation files in the directory to ensure a clean environment for processing.
# This cleans up any files that have "fast" in their names in the subject's anat folder.
[os.remove(f) for f in glob.glob(op.join(preproc_root, 'sub-control01', 'anat', '*fast*'))]

# Define the path where the output of FAST segmentation will be stored.
# The output will have a prefix 'sub-001_T1w_fast' indicating that it is the FAST segmentation result.
segmentation_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_fast')

# Apply
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

Let's check the quality of the segmentation, shall we?
We want to extract 3 tissue types here: the white matter, the grey matter and the csf. How well did fast perform?

In [ ]:
print_dir_tree(bids_root, max_depth=5)

The pve files correspond to our segmented tissues. We have exactly three files, because we set n_classes to 3 above:
```python
fast(..., n_classes=3)


To make it easier on you, we will display:

- pve_0 in <span style="color:red;">red</span>
- pve_1 in <span style="color:green;">green</span>
- pve_2 in <span style="color:blue;">blue</span>

In [ ]:
# Reset any existing overlays in FSLeyes to start with a clean visualization
fsleyesDisplay.resetOverlays()

# Load the skull-stripped T1-weighted image (brain-only) as the base image.
fsleyesDisplay.load(bet_path)

# Load the first partial volume estimate (PVE) map, which typically represents cerebrospinal fluid (CSF).
# The file is identified using a glob pattern that matches files containing 'pve_0' in their name.
fsleyesDisplay.load(glob.glob(op.join(preproc_root, 'sub-control01', 'anat','*pve_0*'))[0])

# Load the second PVE map, which typically represents gray matter (GM).
# The file is identified using a glob pattern that matches files containing 'pve_1' in their name.
fsleyesDisplay.load(glob.glob(op.join(preproc_root, 'sub-control01', 'anat','*pve_1*'))[0])

# Load the third PVE map, which typically represents white matter (WM).
# The file is identified using a glob pattern that matches files containing 'pve_2' in their name.
fsleyesDisplay.load(glob.glob(op.join(preproc_root, 'sub-control01', 'anat','*pve_2*'))[0])

# Set the colormap for the first PVE overlay (CSF) to 'Red'.
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'

# Set the colormap for the second PVE overlay (GM) to 'Green'.
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'

# Set the colormap for the third PVE overlay (WM) to 'Blue'.
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'


# Coregistration of images, a critical preprocessing step

In principle, it could be done manually

In [ ]:
import threading

def launch_freeview(img_list):
    """
    Wrapper around Freeview to launch it with several images.
    This wrapper is necessary to launch Freeview in a separate thread, ensuring the notebook is free to do something else.

    Parameters
    ----------
    img_list: list of string
        List of images (files) to load. Assumed by default to be volume files.
    """
    args = []
    
    # Iterate over the list of images and prepare the command arguments for Freeview
    for i in range(len(img_list)):
        args.append("-v")        # '-v' is used in Freeview to specify the volume files
        args.append(img_list[i]) # Append each image file path to the arguments list

    # Run the Freeview command with all the prepared arguments
    subprocess.run(["freeview"] + args)

# List of images to load in Freeview. 
# img_list contains paths to neuroimaging files you want to visualize:
# A standard anatomical template (MNI152) for reference.
# A subject-specific anatomical image with a specific colormap (greyscale).
imgList = [
    op.expandvars('$FSLDIR/data/standard/MNI152_T1_1mm.nii.gz'), # Path to the standard MNI template image
    op.join(preproc_root, "sub-control01", "anat", "sub-control01_T1w.nii.gz:colormap=greyscale") # Path to the subject's anatomical image with colormap set to greyscale
    # You can modify this list to add any other images you want to view in .nii.gz format
]

# Create a new thread to run Freeview with the list of images
# We pass the target function (launch_freeview) and the arguments (imgList,)
# The comma is necessary to pass a tuple to 'args' so threading works correctly
freeview_thread = threading.Thread(target=launch_freeview, args=(imgList,))

# Start the thread to launch Freeview in the background
freeview_thread.start()

# Print a message indicating that Freeview is running in a separate thread
print("Freeview is running in a separate thread.")


Wait, we can't see anything apart from the top brain. No worries! Simply set the background to be transparent by ticking the Clear Background option, as in the picture below:


### Manual approach

Let's start with a straightforward approach: you will align the images manually. In Freeview, click in Tools > Transform Volume.
You should get the following panel:

Now, play with the sliders of translation and rotation to align the anatomical to the reference. </b> Try to align the two brains as best you can.</p>


The MRI and the template are not very well aligned, but we can try to make them more aligned. Specifically, we would like to find a transformation such that we can align our anatomical to the MNI template. This is the so-called normalization step.

### Types of normalization

So, you now know that you need a transformation and a reference. Great. Now, the transformation you allow can be of two types: it can be linear, meaning whatever you apply will be the same across the entire image, or non linear, where each voxel gets a separate treatment

(ducky linear and non linear)



## Actually doing it: Linear normalization

To perform linear normalization, the idea is simple. The transformation we want should be linear - ie, affine.
Such a matching is usually called in image processing **image registration**. Here, we're dealing with 3D data, so the problem is a bit more complicated. Fortunately all of this has been coded by very smart people, and to our rescue comes a tool specifically to register volumes to each other.

This tool can allow many registrations and is extremely powerful. In its most basic form, it expects:
- An input volume, the volume you want to register (Ducky's sunglasses)
- A reference volume, to which the input is registered (Ducky's body)
- An output volume, the result of the transformation (Ducky's sunglasses once they are on Ducky's beak)

Here is how you can call it to register the patient's anatomical to some reference sitting in another space (here the MNI152 template):
```python
flirt()
```
💡 Pay attention! 💡
    FLIRT expects the anatomical to be skull stripped to maximize normalization. Luckily, you already did it before with BET.</p>


In [ ]:
from fsl.wrappers import flirt
# Import the FLIRT (FMRIB's Linear Image Registration Tool) wrapper from the FSL library to perform image registration.

# The two images
subject_id = 'control01' 
# Define the subject ID. This will be used to specify the path to the subject's anatomical image.

subject_anatomical = op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat', 'sub-control01_T1w')
# Define the path to the subject's T1-weighted anatomical image, located in the preprocessed data directory ('preproc_root') 
# under the subject's anatomical folder ('anat').

mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
# Define the path to the MNI152 standard brain template, which is often used as a reference for alignment.
# The path is constructed using the FSL environment variable '$FSLDIR' and expands it to the full directory path.

###################
# Select which image should be target or reference
# ANSWER:
# The subject anatomical will most often be the target, as the template is usually where we want to map our subjects for 
# group comparison.
# There are cases, however, where we may want the subject anatomical to be the reference. This is the case when we want to map an 
# atlas to a subject while preserving the subject as much as possible.
# We showcase here the case where the subject is chosen as target.
##################

target = subject_anatomical # Set the subject anatomical image as the target image to be aligned.
reference = mni_template # Set the MNI template as the reference image (the space to which the target will be aligned).

result = op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat', 'sub-{}_T1w_mni'.format(subject_id))
# Define the path where the output of the registration will be saved.
# This path includes the subject ID and saves the aligned image in the anatomical folder under 'preproc_root'.
# The output filename includes the suffix '_T1w_mni' to indicate that it is aligned to the MNI space.

flirt(target, reference, out=result)
# Call the FLIRT function to perform the registration:
# - 'target': The image to be transformed (subject anatomical image).
# - 'reference': The image to align to (MNI template).
# - 'out': The path where the registered image will be saved.


Visualize the result of flirt on top of the reference. What do you think of alignment?

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(reference) 
fsleyesDisplay.load(result)

Which cost should we use? If you were in a pure void, there would be no right or wrong answer from the get-go. No choice but to experiment and find out!

Hopefully, <a href="https://fsl.fmrib.ox.ac.uk/fsl/docs/#/registration/flirt/user_guide?id=flirt">the documentation</a> should give you some pointers. What you want here is to register a T1 to a T1: this is a <u>within</u> modality registration, so you should restrict yourself only to costs appropriate to this type of modality! 

To help you, we've set up a cell that will run the different coregistrations for you. Simply fill in the different costs to consider :)

In [ ]:
#######
# Solution
# We consider only within-modalities costs because the two images belong to the same modality (T1-weighted images).
# Therefore, appropriate costs include least squares and normalized correlation ratio.
# Note: Correlation ratio (corratio) can also be used, but it is more commonly applied for different modalities.
#######
possible_costs = ['mutualinfo', 'corratio', 'normcorr', 'normmi', 'leastsq', 'labeldiff']
# Define a list of all possible cost functions supported by FLIRT. These include:
# - 'mutualinfo': Mutual information (cross-modalities).
# - 'corratio': Correlation ratio (can work across or within modalities).
# - 'normcorr': Normalized correlation (within-modalities).
# - 'normmi': Normalized mutual information (cross-modalities).
# - 'leastsq': Least squares (within-modalities).
# - 'labeldiff': Label difference (special cases, such as segmentation images).

costs_to_consider = ['leastsq', 'normcorr'] 
# Select the relevant cost functions for within-modalities (T1-to-T1 alignment):
# - 'leastsq': Uses least squares, suitable for aligning images of the same modality.
# - 'normcorr': Uses normalized correlation, also suitable for within-modality alignment.

# Iterate through the selected cost functions and apply FLIRT for each
for c in costs_to_consider:
    flirt(target, reference, out=result + '_' + c, cost=c)
    # Run FLIRT using the specified cost function:
    # - 'target': The image to be transformed (subject anatomical image).
    # - 'reference': The image to align to (MNI template).
    # - 'out': Save the output file with a suffix based on the cost function used (e.g., '_leastsq' or '_normcorr').
    # - 'cost': The cost function to be used for optimization ('leastsq' or 'normcorr' in this loop).


In [ ]:
for c in costs_to_consider:
    fsleyesDisplay.load(result + '_' + c)

## Non linear normalization

So, you know how to do it linearly.  What if we wanted to do it non-linearly?

With FLIRT, <i>it's painfully hard</i>. To do it, you can use *FNIRT*. 

But, there are other tools available, one of them being <a href="https://github.com/ANTsX/ANTs">ANTs (Advanced Normalization Tools)</a>.
For completeness, we will show you now how to use it (very succinctly) so that you know how to do it.


💡 Pay attention! 💡

FNIRT does NOT expect the input data to be skull-stripped.</p>

In [ ]:
# Read the moving image (subject anatomical image) using the ANTs library.
# The image path is constructed by adding '.nii.gz' to the target path.
# target = subject_anatomical # Set the subject anatomical image as the target image to be aligned.
moving_image = ants.image_read(target + '.nii.gz')

# Read the fixed image (MNI template) using the ANTs library.
# The image path is constructed by adding '.nii.gz' to the reference path.
# reference = mni_template # Set the MNI template as the reference image (the space to which the target will be aligned).

fixed_image = ants.image_read(reference + '.nii.gz')

# Compute the transformation to align the moving image to the fixed image using a non-linear registration method.
# 'SyN' (Symmetric Normalization) is a non-linear transformation method often used for precise alignment.
# The function returns a dictionary containing the transformation details, including the forward transformation.
transformation = ants.registration(fixed=fixed_image, moving=moving_image, type_of_transform='SyN')

# Apply the computed transformation to the moving image to warp it into alignment with the fixed image.
# 'fixed=fixed_image': The reference image that serves as the alignment target.
# 'moving=moving_image': The image to be transformed.
# 'transformlist=transformation['fwdtransforms']': Use the forward transformation obtained from the registration.
warpedImage = ants.apply_transforms(fixed=fixed_image, moving=moving_image, transformlist=transformation['fwdtransforms'])

# Define the output path for the transformed image.
# The path includes the subject ID and indicates that the image has been aligned using the SyN method.
resultAnts = op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat', 'sub-{}_T1w_mni_SyN.nii.gz'.format(subject_id))

# Save the warped image (aligned to the MNI template) to disk using the specified path.
ants.image_write(warpedImage, resultAnts)

# The resulting image can be inspected using FSLeyes or Freeview for visual verification of the alignment.


Look at the results and compare it against the linear coregistration. Which one do you prefer? Why?

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(reference)
fsleyesDisplay.load(result)
fsleyesDisplay.load(resultAnts)


As a final note, all these steps (<u>including</u> non linear normalization!) can be done automatically for you with a single command: <a href="https://web.mit.edu/fsl_v5.0.10/fsl/doc/wiki/fsl_anat.html">fsl_anat</a>. So you might want to use this command, instead of running all of the above when conducting preprocessing.

We provide it here for convenience, but beware: it takes <b>several minutes</b> to complete, so you will need some patience!


In [ ]:
os.system('fsl_anat')

In [ ]:
import shutil

def fsl_anat_wrapped(anatomical_target, output_path):
    # Run the 'fsl_anat' command to perform automatic brain extraction and segmentation on the anatomical image.
    # - 'anatomical_target': Path to the anatomical image to process.
    # - '--clobber': Overwrite any existing output with the same name.
    # - '--nosubcortseg': Skip subcortical segmentation to speed up processing.
    # - '-o': Specifies the output directory where results will be saved.
    os.system('fsl_anat -i {} --clobber --nosubcortseg -o {}'.format(anatomical_target, output_path))
    
    # Define the path to the folder created by FSL ('output_path.anat').
    # FSL saves the output files in this directory by default.
    fsl_anat_path = output_path + '.anat'
    
    # Find all files in the 'output_path.anat' folder.
    files_to_move = glob.glob(op.join(fsl_anat_path, '*'))
    
    # Move each file from the 'output_path.anat' folder to the main 'output_path' folder.
    for f in files_to_move:
        # 'shutil.move' moves each file to the output directory.
        # 'op.split(f)[1]' gets the file name (e.g., 'T1_brain.nii.gz') from the full path.
        shutil.move(f, op.join(output_path, op.split(f)[1]))
    
    # Remove the now-empty 'output_path.anat' directory, as all files have been moved.
    os.rmdir(fsl_anat_path)


In [ ]:
fsl_anat_wrapped(anatomical_path, op.join(preproc_root, 'sub-001', 'anat'))

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Careful ! 💡</b></p>
<p style='text-indent: 10px;'>
    Some of the fMRI preprocessing you will see below leverage the anatomical MRI being preprocessed.
</div>

# fMRI preprocessing

Note that one assumption with fMRI preprocessing is that you've already conducted the anatomical preprocessing. In particular, the two main steps you will need today (repeat it from lab 2) before launching the lab:
- T1 skull-stripping (use BET)
- T1 segmentation (use FAST)

## Problematic volumes removal

To open one particular volume, run the cell below:

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_bold.nii.gz'))

### Field stabilization

The scanner's field takes some time to settle. You probably noticed that the initial volume had a high contrast that quickly decayed to some baseline? It is precisely caused by the scanner's field settling.

These scans are called *non-stationary volumes*, because they are acquired while the B0 field is not yet not stable.

There's little to be done in this regard; we can only throw away the volumes that are contaminated in this specific case, to ensure this change in global signal does not drive our analysis.

If we're being really formal, a higher overall contrast can be detected by looking at the mean voxel value in each volume, like so:

In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib

plt.plot(nib.load(op.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_bold.nii.gz')).get_fdata().mean(axis=(0,1,2)))
plt.xlabel('Time (volume)')
plt.ylabel('Mean voxel intensity')

We'll **choose** to discard the first 10 volumes

Most scanners nowadays actually acquire few scans to help the B0 field settle. These are called dummy scans; the scanner acquires them but throws them away, meaning that you end up with a result that is settled. You should pay attention however if you ever analyze older datasets, as they will not benefit from these new techs obviously!

In [ ]:
### This script considers a test file reported in a lab, feel free to modify it ! ###

# file_to_trim = glob.glob(op.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_bold.nii.gz'))[0]
# Search for the functional MRI (fMRI) file matching the specified pattern using glob.
# The file path is constructed to locate the BOLD image for subject 'sub-01' in the 'ses-test' session under the 'func' directory.
# The '[0]' at the end retrieves the first match found.

# mkdir_no_exist(preproc_root)
# Create the root directory for preprocessed data if it doesn't already exist. This will be used to store the output files.

# mkdir_no_exist(op.join(preproc_root, 'sub-control01'))
# Create a directory for subject 'sub-control01' under the preprocessed data directory if it doesn't exist.

# mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'ses-test'))
# Create a directory for the session 'ses-test' within the subject's directory under the preprocessed data directory.

# mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'ses-test','func'))
# Create the 'func' directory within the session folder. This directory will hold the functional data after processing.

# output_target = op.join(preproc_root, 'sub-control01', 'ses-test', 'func', 'sub-01_ses-test_task-fingerfootlips_bold_settled.nii.gz')
# Define the output path for the trimmed fMRI data. 
# The path includes the subject ID, session ID, and specifies the name for the new file ('sub-01_ses-test_task-fingerfootlips_bold_settled.nii.gz').

############################
# Solution
# We will start from the 10th volume.
# Because indexing starts at 0, the 11th volume corresponds to index 10, so we exclude the first 10 volumes.
# The original dataset has 184 volumes. By excluding the first 10, we retain 174 volumes.
###########################

# Define the variables for the starting volume and the number of volumes to keep
# start_vol = 10 # The volume index to start from (0-based indexing, so this is the 11th volume).
# number_of_volumes = 174 # The number of volumes to keep after trimming, leaving 174 volumes from the original 184.

# fslroi(file_to_trim, output_target, str(start_vol), str(number_of_volumes))
# Run the 'fslroi' command to extract a subset of volumes from the original fMRI file.
# - 'file_to_trim': The path to the original fMRI file.
# - 'output_target': The path where the trimmed file will be saved.
# - 'start_vol': The starting volume (10) to begin extracting.
# - 'number_of_volumes': The number of volumes (174) to extract, starting from 'start_vol'.


## Motion correction

In FSL, we use <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/MCFLIRT">MCFLIRT</a> to perform this correction.

By default, MCFLIRT selects the middle volume of the EPI serie as reference to which other volumes are realigned.</p>


In [ ]:
# If everything already set up, SKIP

from fsl.wrappers import mcflirt

dataset_id = 'ds000171'
subject_id = 'control01' 

# Define the path where the dataset will be stored. Change this to your desired path.
sample_path = "/home/jovyan/Data/dataset"

mkdir_no_exist(sample_path)
# Create the dataset directory if it does not already exist.

bids_root = op.join(os.path.abspath(""), sample_path, dataset_id)
# Define the root directory for the BIDS dataset by combining the current working directory (absolute path),
# the sample path, and the dataset ID.

deriv_root = op.join(bids_root, 'derivatives')
# Define the directory for derivatives (processed files). It is located within the BIDS root.

preproc_root = op.join(bids_root, 'derivatives', 'preprocessed_data')
# Define the directory for preprocessed data, also within the derivatives folder.

subprocess.run([
    "openneuro-py", "download", "--dataset", dataset_id, "--target-dir", bids_root,
    "--include", 'sub-{}'.format(subject_id)
], check=True)
# Download the dataset from OpenNeuro using the openneuro-py command.
# - The dataset ID is specified with '--dataset'.
# - The target directory for the download is set with '--target-dir', which is 'bids_root'.
# - The '--include' flag ensures that only the specific subject (sub-001) is downloaded.
# 'check=True' ensures the command raises an error if it fails.

# Create necessary directories for the derivatives and preprocessed data
mkdir_no_exist(deriv_root) # Create the derivatives directory if it doesn't exist.
mkdir_no_exist(preproc_root) # Create the preprocessed data directory if it doesn't exist.
mkdir_no_exist(os.path.join(preproc_root, 'sub-control01')) # Create the subject directory under preprocessed data.
mkdir_no_exist(os.path.join(preproc_root, 'sub-control01', 'func')) # Create the functional data directory for the subject.


In [ ]:

# Define the path to the original BOLD data file in the BIDS directory structure
path_original_data = os.path.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold')

# Define the path where the motion-corrected BOLD data will be saved
path_moco_data = os.path.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_moco')

mcflirt(
    infile=path_original_data,  # The input file (original BOLD data)
    o=path_moco_data,           # The output file (motion-corrected data)
    plots=True,                 # Generate plots for visualizing motion correction.
    report=True,                # Generate a report on the motion correction process.
    dof=6,                      # Set degrees of freedom to 6 for rigid-body transformation.
    mats=True                   # Save the transformation matrices.
)
# Perform motion correction using MCFLIRT from the FSL library:
# - 'infile' specifies the input BOLD file to correct for motion.
# - 'o' specifies the output file path for the motio


In [ ]:
print_dir_tree(bids_root, max_depth=5) # Print the directory tree of the BIDS root directory up to a maximum depth of 5 levels

In the functional folder, notice that we have two new files:
```
sub-control01_task-music_run-1_bold_moco.nii.gz
sub-control01_task-sitrep_run-1_bold_moco.par

```

The first one is the corrected EPI time serie, with volumes realigned. The second is a file describing the motion parameters that were used to move each volume. It will be useful very shortly to determine which volume moved by a lot.
Notice as well a new directory!
```
sub-control01_task-sitrep_run-1_bold_moco.mat/
```
This directory is full of .MAT files. These are the transformation matrices used for every volume to realign them.

In [ ]:
fsleyesDisplay.resetOverlays() # Reset the overlays in FSLeyes
fsleyesDisplay.load(path_original_data) # Load the original data in FSLeyes
fsleyesDisplay.load(path_moco_data) # Load the motion corrected data in FSLeyes

The motion parameters are stored in the .par file produced by MCFLIRT. Notice that since each volume moved differently, we have one transformation per volume, thus one set of motion parameters per volume as well. We provide you with a way to load these parameters:

In [ ]:
def load_mot_params_fsl_6_dof(path):
    return pd.read_csv(path, sep='  ', header=None, 
            engine='python', names=['Rotation x', 'Rotation y', 'Rotation z','Translation x', 'Translation y', 'Translation z'])

# Load the motion parameters from the motion corrected data. 

mot_params = load_mot_params_fsl_6_dof(op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_moco.par')) 
mot_params # It returns a dataframe with the motion parameters 

Based on **translation on X alone**, can you find perhaps a volume which exceeds with respect to the **preceding volume** a 0.2 mm displacement?

In [ ]:
# write your code here to inspect quickly the translation on X :)
#%matplotlib inline

# Getting the translation is easy
trans_x = mot_params["Translation x"] # Getting the translation on X
# Now, we want a 0.2mm with respect to previous frame:
disp_x = np.diff(trans_x) # Getting the displacement on X. We use np.diff to calculate the difference between consecutive elements in the translation on X.
# Lastly, we can ask for displacements (in absolute value) above 0.2mm and plot it to be clear:
threshold=0.2 
plt.plot(np.abs(disp_x))
plt.hlines(threshold, 0, 370,colors='black', linestyles='dashed', label='FD threshold')
plt.xlabel("Volumes")
plt.ylabel("Framewise translation displacement (mm)")
plt.show()

# There are basically (if looking only along X translation) no frame displacement above 0.2mm! It means that the motion correction was very good for this subject.

Now, we can use the code below to extract an aggregate measure of motion for all volumes. 

In [ ]:
def compute_FD_power(mot_params):
    framewise_diff = mot_params.diff().iloc[1:]

    rot_params = framewise_diff[['Rotation x', 'Rotation y', 'Rotation z']]
    # Estimating displacement on a 50mm radius sphere
    # To know this one, we can remember the definition of the radian!
    # Indeed, let the radian be theta, the arc length be s and the radius be r.
    # Then theta = s / r
    # We want to determine here s, for a sphere of 50mm radius and knowing theta. Easy enough!
    
    # Another way to think about it is through the line integral along the circle.
    # Integrating from 0 to theta with radius 50 will give you, unsurprisingly, r0 theta.
    converted_rots = rot_params*50  
    trans_params = framewise_diff[['Translation x', 'Translation y', 'Translation z']]
    fd = converted_rots.abs().sum(axis=1) + trans_params.abs().sum(axis=1)
    return fd

fd = compute_FD_power(mot_params).to_numpy()

In [ ]:
threshold = np.quantile(fd,0.75) + 1.5*(np.quantile(fd,0.75) - np.quantile(fd,0.25))

In [ ]:
#%matplotlib inline
plt.plot(list(range(1, fd.size+1)), fd)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
plt.hlines(threshold, 0, 370,colors='black', linestyles='dashed', label='FD threshold')
plt.legend()
plt.show()

Okay great, but what if we want to know which volumes are actually above threshold? Simply run the cell below!

In [ ]:
np.where(fd > threshold)[0] + 1 # np.where() returns the indices of the elements that satisfy the condition. 
                                #We add 1 to the result to get the volume numbers instead of the indices. The [0] at the end is to get the array of indices instead of a tuple of arrays.

## Motion-correction: conclusions

Motion correction should always be conducted. As you've seen, it is extremely easy to do and has many benefits. However it is not infaillible. High motion tends to cause non linear effects in the signal that simple motion correction above cannot correct since it has no awareness of the magnetic field. <br>
<br> Motion parameters can, in this case, come to our rescue. As they represent the effect of motion, including them in our modeling to try and correct the signal can help. One could for example include this information in a General Linear Model to regress out the signal of these volumes (censoring) from overall timeseries. ➡️ More on this next week!

## Where are we?

So, let's see what we have done so far:

<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Anatomical</th><td></td><td></td></tr>
    <tr><td></td><td style='text-align:justify;'>Skull stripping</td><td style='text-align:justify;'>Removing skull and surrounding tissues to keep only the brain</td><td style='text-align:justify;'>BET</td></tr>
    <tr><td></td><td style='text-align:justify;'>Segmentation</td><td style='text-align:justify;'>Segmenting brain tissues based on their contrasts</td><td style='text-align:justify;'>FAST</td></tr>
    <tr><td></td><td style='text-align:justify;'>Normalization</td><td style='text-align:justify;'>Mapping participant's brain to a reference brain, making its orientation and scale match so that comparison across participants become feasible.</td><td style='text-align:justify;'>FLIRT</td></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>First few volumes removal</td><td style='text-align:justify;'>Removing volumes for which the B0 field is still not stable and that could contaminate all our data if left unchecked.</td><td style='text-align:justify;'>fslroi</td></tr>
    <tr><td></td><td style='text-align:left;'>Motion correction</td><td style='text-align:justify;'>Realignment of fMRI volumes to a common reference - typically one volume or the average of the volumes - to correct for inter-volume motion. The extracted motion parameters can be used for subsequent analysis (see GLM in two weeks!)</td><td style='text-align:justify;'>MCFLIRT (which is one suboption of FLIRT in fact)</td></tr>
</table>



## Coregistration of functional to anatomical

You have seen coregistration last week, when you were trying to align the T1 to MNI152, both manually and algorithmically. In the specific case of putting a T1 anatomical in a template space (such as MNI), we call it <b>normalization</b>, because we...Normalize it !

You've also seen above with motion-correction coregistration of EPI volumes (functional data) to each other to correct motion.

But what if you wanted to put the functional data overlayed on the anatomy, to know more precisely which parts of the brain are activated? (**coregistration between functional and anatomical**)

Computing the fMRI space to anatomical transformation is precisely the goal of coregistration.
<br><br>
To do this step, we will use : <a href="https://web.mit.edu/fsl_v5.0.10/fsl/doc/wiki/FLIRT(2f)UserGuide.html#epi_reg">epi_reg</a> ! 

### Using epi_reg to do the EPI registration



<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Pay attention ! 💡</b></p>
<p style='text-indent: 10px;'>
    Make sure that the whole head T1 and the skull-stripped T1 have the same orientation.
For example, if you ran fsl_anat to extract the brain (which is fine), FSL will change in the headers the orientation of the T1 before skull-stripping. As a consequence, the brain-extracted T1 no longer has the same orientation as the original T1. If you display them on top of each other, they are perfectly matched, but not from the perspective of the <b>headers</b>, which can play nasty tricks on you when performing coregistration.</p>
</span>
</div>

Note that if parallel acceleration is used in the EPI acquisition then the *effective echo spacing* is the actual echo spacing between acquired lines in k-space divided by the acceleration factor.

epi_reg has one peculiarity. If you launch it on a 4D volume, it will truncate your result to the first volume, because it expects a *single* EPI volume. We should thus first extract a single volume from our EPI, and then call epi_reg on it. We do that for you below.

If you want to run with 4D volume to see the result and the warning, set use_single_vol to False

In [ ]:
from fsl.wrappers import epi_reg

#################
# Solution
# We use the motion-corrected EPI
##################

# Define the path to the motion-corrected EPI (functional) image.
epi_target = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_moco') # To change 

# Define the path to the whole (unprocessed) T1-weighted anatomical image.
whole_t1 = op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w')

# Define the path to the skull-stripped T1-weighted anatomical image.
skull_stripped_t1 = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w') # ...

# Define the path where the output of the epi_reg process will be saved.
output_path = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_bbr') # ....

# Define the path for the middle volume of the motion-corrected EPI file, which will be extracted as a reference.
ref_vol_name = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_moco_vol_middle') 

# Set whether to use a single volume as a reference. This can be useful for registration purposes.
use_single_vol = True

if use_single_vol:
    # Extract the middle volume of the EPI image using fslroi. This extracts a single volume from the time series.
    # '182' is the index of the middle volume (assuming the total number of volumes is known). # To change 
    # '1' indicates that only one volume will be extracted.
    fslroi(epi_target, ref_vol_name, str(), str(1))
    
    # Run epi_reg to register the extracted middle EPI volume to the T1-weighted anatomical image.
    subprocess.run([
        'epi_reg',
        '--epi={}'.format(ref_vol_name),  # The EPI image (middle volume) to be registered.
        '--t1={}'.format(whole_t1),       # The whole, non-skull-stripped T1-weighted image.
        '--t1brain={}'.format(skull_stripped_t1),  # The skull-stripped T1-weighted image.
        '--out={}'.format(output_path)    # The output path where the registered image will be saved.
    ])
else:
    # If not using a single volume, register the entire motion-corrected EPI series to the T1-weighted anatomical image.
    subprocess.run([
        'epi_reg',
        '--epi={}'.format(epi_target),    # The full motion-corrected EPI image (all volumes).
        '--t1={}'.format(whole_t1),       # The whole, non-skull-stripped T1-weighted image.
        '--t1brain={}'.format(skull_stripped_t1),  # The skull-stripped T1-weighted image.
        '--out={}'.format(output_path)    # The output path where the registered image will be saved.
    ])


Notice how FAST is ran?
This is because the specific coregistration cost (boundary-based registration, BBR) uses the **anatomical white-matter tissues from FAST**. If no such tissue is provided to the function, it re-runs FAST to obtain it and use it. If you've already done anatomical segmentation. 

If you had to yourself correct the white matter with the help of an expert because somehow FSL did a poor job on your data. Clearly you'd like to have this one used instead of the result from FAST, right?

Well- you can! We just need a new option in the epi_reg command:
```python
epi_reg(...,wmseg=path_to_your_white_matter_segmentation)
```
Remember from last week, which T1 file corresponds to the white matter, between pve_0, pve_1 and pve_2 ? 

In [ ]:
###############
# Solution
# White matter corresponds to pve_2.
##############

white_matter_segmentation = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_fast_pve_2.nii.gz') # We provide the white matter segmentation

subprocess.run(['epi_reg','--epi={}'.format(ref_vol_name), '--t1={}'.format(whole_t1), '--t1brain={}'.format(skull_stripped_t1), 
                '--out={}'.format(output_path),
               '--wmseg={}'.format(white_matter_segmentation)])

Let's overlay the two (EPI and anatomical) on top of each other to visualize the quality of the coregistration!

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(skull_stripped_t1)
fsleyesDisplay.load(output_path)


- Is the functional in the right orientation?
- Are the ventricles correctly aligned?
- Are the boundaries of the EPI more or less matching the anatomical?

➡️ You can also check how the white matter of the EPI matches your anatomical's white matter provided you have sufficient resolution

#### Some cleanup
If you have a look, you might notice that perhaps your directory got filled with many files. These are temporary files, created but uncorrectly not eliminated by epi_reg. The following should help:


In [ ]:
def cleanup_epi_reg(path_to_clean):
    patterns = ['*_fast_*', '*_fieldmap*']
    for p in patterns:
        files = glob.glob(op.join(path_to_clean, p))
        for f in files:
            os.remove(f)

In [ ]:
cleanup_epi_reg(op.join(preproc_root, 'sub-control01', 'func'))

In [ ]:
print_dir_tree(bids_root, max_depth=5)

#### Smoothing

All these transforms are not exactly perfect. As you've seen in class, a step of smoothing is typically applied, with the size of the smoothing being dependent on your application, starting resolution etc.
The idea of smoothing is really that, as you're averaging, hopefully you increase the signal to noise ratio. <br>
A side-effect is that finest patterns of activation will be lost in the averaging (we can't have everything: there's no free lunch).

With FSL, smoothing is rather easy to do. However, one thing which is important is the size of your filter.
Different softwares might use different conventions. For MRI, it is typical to talk about FWHM (Full-width at half maximum), expressed in mms.

FSL, however, takes as input in sigma instead of FWHM. The conversion is easy fortunately:

$$ \sigma = \frac{FWHM}{2.3548}$$

Here for example would be the smoothing command for 6mm FWHM smoothing:

In [ ]:
# Define the path where the output will be saved (same as input).
# output_path = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_bbr') # ....

cmd = 'fslmaths {} -s {} {}_smoothed-6mm'.format(output_path, 6/2.3548, output_path)
# This line constructs a command string using 'fslmaths' to apply spatial smoothing to the image.
# - 'fslmaths' is the FSL command used for mathematical manipulation of images.
# - '{}': Placeholder for 'output_path', which is the path to the image that needs smoothing.
# - '-s': This option applies a Gaussian kernel smoothing with a given standard deviation (sigma).
# - '6/2.3548': The value (6 mm divided by 2.3548) converts the full width at half maximum (FWHM) to sigma.
# - '{}_smoothed-6mm': This specifies the output file name, indicating that it has been smoothed with a 6 mm FWHM kernel.

subprocess.run(['fslmaths', output_path, '-s', str(6/2.3548), '{}_smoothed-6mm'.format(output_path)])
# This line runs the 'fslmaths' command directly as a subprocess.
# - 'fslmaths': The command to manipulate the image.
# - 'output_path': The input file path of the image to be smoothed.
# - '-s': The option for applying Gaussian smoothing.
# - 'str(6/2.3548)': The sigma value calculated by converting the 6 mm FWHM value to standard deviation.
# - '{}_smoothed-6mm'.format(output_path)': The output path where the smoothed image will be saved, named with a suffix '_smoothed-6mm'.




Let's observe what we have now:

In [ ]:
fsleyesDisplay.load(output_path + '_smoothed-6mm')

## 1.7 MRI + fMRI preprocessing: summary

So, these were all the steps you were meant to study this week. Next week, we'll present advanced steps of fMRI, but they are not always conducted unlike those we've shown you above which should always be considered.

You should know by now: preprocessing is extremely important and you will likely spend a lot of time on it. Decisions in preprocessing will affect your analysis, so do not take this step lightly, it is <u>critical</u> to do it as well as possible!

<u>Always perform quality control to ensure everything is okay!</u>

Let's review one last time the different steps you've studied and which FSL tool(s) you used to do it:
<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Anatomical</th><td></td><td></td></tr>
    <tr><td></td><td style='text-align:justify;'>Skull stripping</td><td style='text-align:justify;'>Removing skull and surrounding tissues to keep only the brain</td><td style='text-align:justify;'>BET</td></tr>
    <tr><td></td><td style='text-align:justify;'>Segmentation</td><td style='text-align:justify;'>Segmenting brain tissues based on their contrasts</td><td style='text-align:justify;'>FAST</td></tr>
    <tr><td></td><td style='text-align:justify;'>Normalization</td><td style='text-align:justify;'>Mapping participant's brain to a reference brain, making its orientation and scale match so that comparison across participants become feasible.</td><td style='text-align:justify;'>FLIRT + FNIRT (from last week), or ANTs</td></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>First few volumes removal</td><td style='text-align:justify;'>Removing volumes for which the B0 field is still not stable and that could contaminate all our data if left unchecked.</td><td style='text-align:justify;'>fslroi</td></tr>
    <tr><td></td><td style='text-align:left;'>Motion correction</td><td style='text-align:justify;'>Realignment of fMRI volumes to a common reference - typically one volume or the average of the volumes - to correct for inter-volume motion. The extracted motion parameters can be used for subsequent analysis (see GLM next week!)</td><td style='text-align:justify;'>MCFLIRT (which is one suboption of FLIRT in fact)</td></tr>
    <tr><td></td><td style='text-align:left;'>Coregistration to anatomical</td><td style='text-align:justify;'>Putting the functional volumes in anatomical space</td><td style='text-align:justify;'>FLIRT (epi_reg being a specialized instance)</td></tr>
    <tr><td></td><td style='text-align:left;'>Smoothing</td><td style='text-align:justify;'>Allowing a bit of lee-way in the voxel's values to account for the imperfection of the registration</td><td style='text-align:justify;'>fslmath with smoothing operation</td></tr>
</table>

### Advanced preprocessing 

## Field map unwarping

The field itself is not homogeneous. This means, in turn, that there are distortions in the acquisition.
We can try to correct for it, through field maps, provided they've been acquired.

### What are field maps ? 

Field maps are maps of the magnetic field (hence their name). They are acquired during an experimental session to capture parts where the MRI's magnetic field might present inhomogeneities. These inhomogeneities will, in turn, cause distortions in the signal which are not part of the subject's anatomy, as well as signal drop (places where the contrast becomes very small between tissues). Such artefacts should obviously be removed.
This is where fieldmaps are typically coming into play: by knowing how your scanner is distorting your signal, you can hope to correct for it - to some amount.

The first step is - naturally - to acquire fieldmaps. You will need to download them as we have avoided loading them for you - on purpose!

To make sure you've understood how to load datasets, here is the dataset of interest: https://openneuro.org/datasets/ds004226/versions/1.0.0


Your first task is to load:
- Subject 001 data files, including the fieldmap files, located in the fmap subfolder (WITH the JSON sidecars!)



In [ ]:
# If everything set up, SKIP it!

import subprocess

dataset_id = 'ds000171'
subject_id = 'control01'

sample_path = "/home/jovyan/Data/dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

fmap_path = op.join(bids_root, 'sub-001', 'fmap') 
subject_dir = 'sub-{}'.format(subject_id)

##################
# Solution
# There are two solutions
# The easiest is simply to include all data of subject-01
# The other is to add one line for the fieldmaps
##################
# Change the command below to include files in the fmap subdirectory
# You should STILL be loading the EPI and anatomical
subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, # Openneuro has for each dataset a unique identifier
                "--target-dir", bids_root,  # The path where we want to save our data. You should save your data under /home/jovyan/Data/[your dataset ID] to be 100% fool-proof
                "--include", op.join(subject_dir, 'anat','*'),# We are asking to get all files within the subject_dir/anat folder by using the wildcard *
                "--include", op.join(subject_dir, 'func','*'),# We are asking to get all files within the subject_dir/func folder by using the wildcard *
                "--include", op.join(subject_dir, 'fmap','*'),# We are asking to get all files within the subject_dir/fmap folder by using the wildcard *
               ], check=True)

# Simple variant to include everything from subject-01
subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, # Openneuro has for each dataset a unique identifier
                "--target-dir", bids_root,  # The path where we want to save our data. You should save your data under /home/jovyan/Data/[your dataset ID] to be 100% fool-proof
                "--include", subject_dir # Effectively get all data
               ], check=True)

Again, remember this download might not be finished immediately. Now, assuming it *is*, let's have a look at what we have:

In [ ]:
print_dir_tree(bids_root, max_depth=5)

Look in the fmap folder. There are four files, corresponding to two fieldmap acquisitions. One is PA, the other is AP.

We need some parameters to be able to exploit these files.
In particular, we need to figure out:
- The phase encoding direction
- The total readout time



In [ ]:
get_json_from_file(op.join(fmap_path, 'sub-control01_acq-task_dir-{}_epi.json'.format('AP')))

In [ ]:
# If everything is already set up, skip it!

mkdir_no_exist(deriv_root)
mkdir_no_exist(preproc_root)
mkdir_no_exist(op.join(preproc_root, 'sub-control01'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'anat'))
mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'fmap'))

In [ ]:
preproc_fmap_path = op.join(preproc_root, 'sub-control01', 'fmap')
# Define the path for storing the preprocessed fieldmap data for the subject 'sub-001' within the preprocessed data directory.

mkdir_no_exist(preproc_fmap_path)
# Create the directory if it doesn't already exist. This directory will hold the files related to the fieldmaps.

direction_file = op.join(preproc_fmap_path, 'datain.txt')
# Define the path for the 'datain.txt' file, which will store information about the phase encoding directions and total readout times.

f = open(direction_file, 'w')
# Open the 'datain.txt' file in write mode. This file will be used to record phase encoding direction information for the fieldmaps.

for name in ['AP', 'PA']:
    # Iterate over the two phase encoding directions: 'AP' (Anterior-Posterior) and 'PA' (Posterior-Anterior).
    
    data = get_json_from_file(op.join(fmap_path, 'sub-control01_acq-task_dir-{}_epi.json'.format(name)))
    # Read the JSON file for the fieldmap corresponding to the current direction ('AP' or 'PA').
    # These JSON files contain metadata about the fieldmaps, such as phase encoding direction and total readout time.
    
    phase_dir = data['PhaseEncodingDirection'] # Extract the phase encoding direction from the JSON data.
    total_readout_time = data['TotalReadoutTime'] # Extract the total readout time from the JSON data.

    # We expect a specific format: x y z total_readout_time.
    # The values x, y, and z are set to 1/-1 if they match the phase encoding direction, otherwise they are set to 0.
    phase = [0, 0, 0, total_readout_time] # Initialize the phase list with zeros for x, y, and z, and the total readout time as the fourth element.
    
    is_neg = len(phase_dir) == 2 and phase_dir[1] == '-'
    # Check if the phase encoding direction is negative (e.g., 'j-' or 'i-'). If it is negative, set 'is_neg' to True.

    phase_dir = phase_dir[0]
    # Extract the first character ('i', 'j', or 'k') from the phase encoding direction, which represents the axis (x, y, or z).

    phase[ord(phase_dir) - ord('i')] = -1 if is_neg else 1
    # Determine which axis ('i' for x, 'j' for y, or 'k' for z) the phase encoding applies to by subtracting the ASCII value of 'i' from 'phase_dir'.
    # Set the corresponding element in the phase list to -1 if it's a negative direction, or 1 otherwise.

    for i in range(3):
        f.write('{} {} {} {}\n'.format(phase[0], phase[1], phase[2], phase[3]))
        # Write the phase encoding information to the 'datain.txt' file in the format: x y z total_readout_time.
        # Repeat this three times for each phase encoding direction ('AP' and 'PA').

f.close()
# Close the 'datain.txt' file after writing all the information.


### Creating the field map
Now, we will create the field map.
This process is tedious, sometimes hard to get right. First, let's look at the two fieldmaps.

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(fmap_path, 'sub-control01_acq-task_dir-AP_epi.nii.gz'))
fsleyesDisplay.load(op.join(fmap_path, 'sub-control01_acq-task_dir-PA_epi.nii.gz'))

As you notice, they are quite different with respect to their distortions. This is because they used different phase encoding directions (Anterior -> Posterior and Posterior -> Anterior, hence AP and PA).

Looking from these two encoding directions, we (or rather clever algorithm: <a href="https://web.mit.edu/fsl_v5.0.10/fsl/doc/wiki/topup.html">topup</a>) can build a complete map of the distortions on our EPI / fMRI file.

First, AP and PA fieldmaps should be made a single file to be fed to topup.
- You also need to feed it the direction file we created above (which specifies encoding direction and readout time of our EPI).
- From the computed field, we need to convert it to radians, and finally, we obtain both a phase information for the field - in radians - and a magnitude information.

You are now ready to apply the fieldmap to correct distortions.

The function below conducts all these steps:

In [ ]:
os.system("topup")

In [ ]:
# Recall:  direction_file = op.join(preproc_fmap_path, 'datain.txt')

def generate_fmap_AP_PA(direction_file):
    """
    From an AP/PA pair of files, generate the corresponding fieldmap files.

    Parameters
    ----------
    direction_file: str
        Path to a direction file (typically called datain.txt) indicating phase encoding direction, total 
        readout time, and other relevant parameters for fieldmap computation.
    """
    merged_phase_imgs = op.join(preproc_fmap_path, 'sub-control01_acq-task_dir-fmap_merged')
    # Define the path for the merged AP and PA phase images. This file will store the combined EPI images from both directions.

    # Step 1: Combine AP and PA as a single file
    subprocess.run(['fslmerge', '-t', merged_phase_imgs, 
                    op.join(fmap_path, 'sub-control01_acq-task_dir-AP_epi.nii.gz'), 
                    op.join(fmap_path, 'sub-control01_acq-task_dir-PA_epi.nii.gz')])
    # Merge the AP and PA phase images using 'fslmerge'. 
    # - '-t' specifies merging along the time axis (creates a 4D file with both volumes).
    # - The output file is 'merged_phase_imgs', which combines the AP and PA files.

    # Step 2: Compute the fieldmap deformation with topup
    output_fmap = op.join(preproc_fmap_path, 'fieldmap_ex')
    # Define the path for the output fieldmap (deformation map).

    unwarped_img = op.join(preproc_fmap_path, 'se_epi_unwarped')
    # Define the path for the output unwarped EPI image after correction.

    subprocess.run(['topup', 
                    '--imain={}'.format(merged_phase_imgs), 
                    '--datain={}'.format(direction_file),
                    '--config={}'.format('b02b0.cnf'),
                    '--fout={}'.format(output_fmap),
                    '--iout={}'.format(unwarped_img),
                    '-v'])
    # Run the 'topup' command to calculate the deformation field for the fieldmap.
    # - '--imain': Specifies the input merged AP and PA EPI images.
    # - '--datain': The direction file containing phase encoding directions and readout times ('datain.txt').
    # - '--config': The configuration file ('b02b0.cnf') specific for fieldmap computation.
    # - '--fout': The output deformation field file ('output_fmap').
    # - '--iout': The output unwarped image ('unwarped_img').
    # - '-v': Enables verbose output for detailed logs.

    # Step 3: Convert fieldmap units to radians
    subprocess.run(['fslmaths', output_fmap, '-mul', str(6.28), output_fmap + '_rads'])
    # Convert the fieldmap values to radians using 'fslmaths'.
    # - Multiply the fieldmap values by 2 * pi (6.28) to convert the output into radians.
    # - Save the result as 'output_fmap_rads'.

    # Step 4: Create a magnitude fieldmap
    subprocess.run(['fslmaths', unwarped_img, '-Tmean', output_fmap + '_mag'])
    # Create a magnitude image from the unwarped EPI image using 'fslmaths'.
    # - '-Tmean' computes the mean across the time axis (if the image is 4D).
    # - The result is saved as 'output_fmap_mag'.

    # Extract fieldmap brain using BET (Brain Extraction Tool)
    subprocess.run(['bet', output_fmap + '_mag', output_fmap + '_mag_brain', '-m', '-R'])
    # Run 'bet' to extract the brain from the magnitude image.
    # - The input is 'output_fmap_mag', and the output is 'output_fmap_mag_brain'.
    # - '-m' outputs a brain mask as well.
    # - '-R' uses a robust brain center estimation.

# Call the function to generate the fieldmap using the specified direction file
generate_fmap_AP_PA(direction_file)


It now remains to apply the fieldmap! 

To do so we will apply to the **first volume of our series to show you the result of distortion correction**.
Therefore, extract the first volume:

In [ ]:
file_to_trim = op.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold.nii.gz')
# This line creates the file path to the specific EPI (fMRI) volume we want to trim. 
# It uses the 'op.join' function to concatenate the root directory ('bids_root') with the sub-directory and file name.
# The file ('sub-control01_task-music_run-1_bold.nii.gz') is located in the 'func' folder for subject 'sub-control01'.

mkdir_no_exist(op.join(preproc_root, 'sub-control01', 'func'))
# This line creates a new directory in the preprocessed root path ('preproc_root') for the same subject ('sub-control01') if it does not already exist.
# The new directory will store the processed files. The function 'mkdir_no_exist' ensures the directory is created only if it doesn't already exist.

extracted_epi = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run_1_bold_vol_1')
# This line sets the file path for the output volume after extraction. 
# It uses 'op.join' to concatenate 'preproc_root' with the necessary sub-directory and file name where the first volume will be saved.
# The output file is named 'sub-control01_task-music_run_1_bold_vol_1'.

###########
# Solution
# Extracting the first volume means we keep a single volume. 
# This is exactly as we did last week for the reference EPI used in epi_reg,
# but we take the volume number 0 instead of the middle one.
###########

# Select only the FIRST volume!
start_vol = 0 # Where should we start? (First volume is 0, not 1!)
# This variable defines the starting volume index for extraction. In fMRI data, indexing typically starts from 0.
# Here, '0' indicates that we want to extract the very first volume in the 4D EPI data.

number_of_volumes = 1 # How many volumes should we keep?
# This variable defines the number of volumes to extract from the dataset. 
# Setting this to '1' means that only the first volume will be extracted.

fslroi(file_to_trim, extracted_epi, str(start_vol), str(number_of_volumes))
# This line calls 'fslroi', a command-line tool from FSL (FMRIB Software Library), to extract a specific subset of volumes from the 4D EPI file.
# 'file_to_trim' is the input EPI file, 'extracted_epi' is the path where the extracted volume will be saved.
# The function uses 'start_vol' to determine where to start the extraction and 'number_of_volumes' to determine how many volumes to keep.
# This command extracts the first volume (volume 0) from the EPI data and saves it as a new file.


In [ ]:
fsleyesDisplay.load(extracted_epi)

We are finally able to apply our fieldmap to the EPI. We can do so, using FUGUE!

In [ ]:
fmap_path_rad = op.join(preproc_root, 'sub-control01', 'fmap', 'fieldmap_ex_rads')
# Define the path to the fieldmap file in radians. This file was created in a previous step and is stored in the 'fmap' directory for subject 'sub-001'.

epi_result = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_vol_1_fmap')
# Define the output path for the unwarped EPI volume. The result will be saved as 'sub-001_task-sitrep_run-01_bold_vol_1_fmap' in the 'func' directory.

unwarpdir = 'y-'
# Define the direction for the unwarping. Here, 'y-' indicates that the phase encoding direction is along the negative y-axis.

dwell_time = data['EffectiveEchoSpacing']
# Extract the dwell time (Effective Echo Spacing) from the dataset's metadata.
# Note: The actual dwell time in the image header might be incorrect (reported in microseconds instead of milliseconds). 
# The 'EffectiveEchoSpacing' provides the correct magnitude and is used instead.

# Apply the fieldmap correction using FUGUE (FSL utility for unwarping EPI images):
subprocess.run([
    'fugue', 
    '-i', extracted_epi,  # The input EPI volume (the one that will be unwarped).
    '--loadfmap={}'.format(fmap_path_rad),  # Load the fieldmap (in radians) generated earlier.
    '--dwell={}'.format(dwell_time),  # Set the dwell time parameter for accurate unwarping.
    '--unwarpdir={}'.format(unwarpdir),  # Specify the direction of phase encoding for unwarping.
    '-u', epi_result  # The output path for the unwarped EPI image.
])


Let's visualize!

In [ ]:
fsleyesDisplay.load(epi_result)

<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'>
    Field maps for this dataset came in a specific format. But they can come in <b>many</b> different ways, meaning you will need to be very careful when recovering them. The steps outlined above in particular are only applicable in the case of having an AP-PA acquisition. Here is the full resource of FSL's FUGUE on field map unwarping: <a href=https://fsl.fmrib.ox.ac.uk/fsl/docs/#/registration/fugue>https://fsl.fmrib.ox.ac.uk/fsl/docs/#/registration/fugue</a> . Don't be afraid to refer to it, should you have a different format in a project!</p>
</span>
</div>


## Combining transforms

We know how to perform motion-correction, and how to coregister an image to another. That's great!
But each transformation usually implies a step of interpolation (because the image is transformed and must be resampled). This interpolation means the resulting data is "corrupted" slightly. We would like to minimize the amount of interpolations to only once if possible.


The order of transformations we would like to have is:
EPI → Motion correction (Mcflirt) → EPI (motion corrected) → Field unwarping + affine coregistration (epi_reg with fieldmap) → EPI (Anatomical space) → Normalization (Flirt or ANTs) → EPI (template space)

## Combining fieldmap unwarping and EPI registration

FSL provides a way to compute the EPI to anatomical while combining it with the fieldmap unwarping. 

But before applying all transforms, let's worry about doing the required preprocessing, namely:
- Motion correction
- Field unwarping
- EPI to anatomical coregistration
- Anatomical to template coregistration (Normalization): we will use the MNI152 1mm template that you know from the previous weeks

First, remark that motion correction is done by selecting a reference volume in the EPI to which all others are coregistered. By default, the middle EPI was used. Because we used in our fieldmap computation the first EPI, we need to use this one instead.

**It is critical that you pay attention to which image was used to compute your transformations, otherwise combining them won't make sense!**. For this reason, let's now go over the entire pipeline and transformation steps, sticking to the first EPI. We extract it again with fslroi, and we re-run the motion correction.

In [ ]:
original_epi = op.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold')
# Define the path to the original EPI (BOLD) image for subject 'sub-001'.
# This file is located in the 'func' directory under the BIDS root directory.

reference_epi = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_first-vol')
# Define the output path for the extracted first volume of the EPI image.
# This file will be saved in the preprocessed directory under the same 'func' directory as the original EPI file.

fslroi(original_epi, reference_epi, str(0), str(1))
# Use the FSL tool 'fslroi' to extract the first volume from the original EPI image:
# - 'original_epi': The path to the input EPI image (the full 4D dataset).
# - 'reference_epi': The output path where the first volume will be saved.
# - 'str(0)': The index of the volume to start extracting from (0 indicates the first volume).
# - 'str(1)': The number of volumes to extract (1 means only the first volume will be extracted).


Now, let's do **motion correction**. Recall that it is done on the **entire** EPI timeseries with mcflirt. We will explicitly give the first epi as reference this time around, to force FSL to use this volume and realign everyone to it!

In [ ]:
from fsl.wrappers import mcflirt

# Define the path for the motion-corrected output file.
path_moco_data = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_moco')
# This is where the motion-corrected EPI (BOLD) image will be saved after running MCFLIRT.
# The file will be stored in the 'func' directory within the preprocessed data directory for subject 'sub-001'.

# Run the MCFLIRT command to perform motion correction on the EPI image.
mcflirt(
    infile=original_epi,          # The input 4D EPI dataset that needs motion correction.
    o=path_moco_data,             # The output file path where the motion-corrected data will be saved.
    plots=True,                   # Generate motion correction plots for quality assessment.
    report=True,                  # Generate a report that summarizes the motion correction process.
    dof=6,                        # Use 6 degrees of freedom (rigid-body motion correction: 3 translations + 3 rotations).
    mats=True,                    # Save the transformation matrices for each volume.
    reffile=reference_epi         # Use the first volume extracted (reference_epi) as the reference image for motion correction.
)


Fantastic! Now, because **the reference volume did not move at all** (since it is the reference to which everyone is realigned), we can use this volume as starting point to compute our other transforms: we're only missing the coregistration with fieldmap unwarping, as the normalization is obtained through the anatomical data :)


In [ ]:
subject_id = 'control'
subject = 'sub-control01'

# Relevant paths for anatomical preprocessing
anatomical_path = op.join(bids_root, subject, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id))
# Path to the original T1-weighted anatomical image for the subject.

betted_brain_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w'.format(subject_id))
# Path where the skull-stripped version of the anatomical image will be saved after using BET.

segmentation_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_fast')
# Path where the segmented brain tissues (gray matter, white matter, etc.) will be saved using FAST.

mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
# Path to the MNI152 template brain image. This is used for normalization.

anat_result = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w_mni'.format(subject_id))
# Path where the normalized anatomical image (aligned to MNI space) will be saved.

anat_2_mni_trans = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w_2_mni_lin.mat'.format(subject_id))
# Path where the linear transformation matrix from the subject's anatomical space to MNI space will be saved.

# Relevant variables for epi_reg
output_path = op.join(preproc_root, 'sub-001', 'func', 'sub-control01_task-music_run-1_bold_anat-space')
# Path where the EPI image registered to anatomical space will be saved.

dwell_time = 0.000620007
# Dwell time for the EPI image, which is needed for fieldmap unwarping.

unwarpdir = 'y-'
# Unwarping direction for fieldmap correction, indicating the phase encoding direction.

##############
# Steps needed before using epi_reg:
# - Brain extraction with BET
# - Segmentation with FAST
# - Normalization to MNI152 space with FLIRT
##############

from fsl.wrappers import fast

# Perform brain extraction using BET
subprocess.run(['bet', anatomical_path, betted_brain_path, '-m', '-R'])
# 'bet': Brain extraction tool
# 'anatomical_path': Input anatomical image
# 'betted_brain_path': Output skull-stripped image
# '-m': Generate a brain mask
# '-R': Robust estimation of brain center

# Perform tissue segmentation using FAST
fast(imgs=[betted_brain_path], out=segmentation_path, n_classes=3)
# FAST is used for segmentation of the skull-stripped image
# 'imgs': Input image (skull-stripped)
# 'out': Output path for segmentation results
# 'n_classes': Number of tissue classes (3: gray matter, white matter, CSF)

# Normalize the skull-stripped anatomical image to MNI space using FLIRT
flirt(betted_brain_path, mni_template, out=anat_result, omat=anat_2_mni_trans)
# 'flirt': Linear registration tool
# 'betted_brain_path': Skull-stripped anatomical image
# 'mni_template': MNI template for alignment
# 'out': Output path for the aligned anatomical image
# 'omat': Output path for the transformation matrix

#############
# Perform EPI-to-anatomical registration with fieldmap correction using epi_reg
#############
reference_epi = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_first-vol')
# Path for the first volume of the EPI image, used as a reference for registration.

# Extract the first volume of the EPI dataset
fslroi(original_epi, reference_epi, str(0), str(1))
# 'fslroi': Extracts a single volume from the 4D EPI image
# 'original_epi': Input EPI image
# 'reference_epi': Output path for the first volume
# 'str(0)': Index of the first volume (0-based)
# 'str(1)': Number of volumes to extract (1)

# Run epi_reg to perform EPI-to-anatomical registration with fieldmap unwarping
subprocess.run([
    'epi_reg',
    '--epi={}'.format(reference_epi),  # Input EPI volume
    '--t1={}'.format(anatomical_path),  # Input T1-weighted image
    '--t1brain={}'.format(betted_brain_path),  # Skull-stripped T1-weighted image
    '--out={}'.format(output_path),  # Output path for the registered EPI image
    '--fmap={}'.format(op.join(preproc_root, 'sub-control01', 'fmap', 'fieldmap_ex_rads')),  # Fieldmap in radians
    '--fmapmagbrain={}'.format(op.join(preproc_root, 'sub-control01', 'fmap', 'fieldmap_ex_mag_brain')),  # Skull-stripped magnitude image
    '--fmapmag={}'.format(op.join(preproc_root, 'sub-control01', 'fmap', 'fieldmap_ex_mag')),  # Magnitude image of the fieldmap
    '--wmseg={}'.format(op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_fast_pve_2')),  # White matter segmentation
    '--echospacing={}'.format(dwell_time),  # Echo spacing (dwell time) for fieldmap correction
    '--pedir={}'.format(unwarpdir)  # Phase encoding direction
])

print("Done with EPI to anatomical registration with fieldmap unwarping")

###### Note:  the code works on a single volume of the EPI dataset. ######


## Summary

The code performs several key preprocessing steps for aligning and correcting functional (EPI) MRI images with anatomical (T1-weighted) MRI images:
### Anatomical Preprocessing:
- Skull-Stripping: Uses BET (Brain Extraction Tool) to remove non-brain tissue from the T1-weighted anatomical image.
- Segmentation: Applies FAST (FMRIB's Automated Segmentation Tool) to segment the skull-stripped image into different tissue types (e.g., gray matter, white matter, CSF).
- Normalization: Uses FLIRT (FMRIB's Linear Image Registration Tool) to align the anatomical image to the MNI152 standard space, saving the transformation matrix for potential later use.
### EPI Preprocessing:
- Extracts the first volume from the 4D EPI dataset to use as a reference for alignment.
- Applies epi_reg to register the EPI image to the anatomical space using the skull-stripped T1 image. It also incorporates fieldmap correction to correct for distortions based on phase encoding direction and echo spacing information.

➡️ Inspect the two resulting files to ensure that nothing went wrong. In other words:
- Check that the MCFLIRT result is okay with respect to motion
- Check that the realignment following epi_reg made the EPI well aligned with the anatomical

### Getting the saved transformation

Applying the transformation to a single volume is nice, but we should still need to know where the transformation was saved, to apply it to all other volumes of interest.

Let's inspect our resulting folders

In [ ]:
print_dir_tree(bids_root,max_depth=5)

The file <b>sub-001_task-sitrep_run-01_bold_anat-space_warp</b> corresponds to the <u>transformation</u> from EPI to anatomical file with the fieldmap unwarping applied.

### Combining the transforms

Let's list our available transforms as is:

 <table>
  <tr>
    <th>Transform</th>
    <th>Step</th>
    <th>File(s)</th>
    <th>Linear?</th>
  </tr>
  <tr>
    <td>Each EPI to reference EPI</td>
    <td>Motion correction (mcflirt) </td>
    <td>matrices in func/sub-001_task-sitrep_run-01_bold_moco.mat/</td>
    <td>Yes</td>
  </tr>
  <tr>
    <td>EPI to anat (with fieldmap)</td>
    <td>Functional to anat coregistration (epi_reg) </td>
    <td>func/sub-001_task-sitrep_run-01_bold_anat-space_warp.nii.gz</td>
    <td>No</td>
  </tr>
  <tr>
    <td>Anat to template</td>
    <td>Normalization (FLIRT or FNIRT or ANTs)</td>
    <td>anat/.mat</td>
    <td>Yes if FLIRT or ANTs with linear transform, no otherwise</td>
  </tr>
</table> 

We will now combine all these transformations, so that we apply one transformation and exactly one interpolation at the end to minimize errors.

We provide you with a function to do so just below, along with a function to apply the resulting combined transformation. Beware, all these functions operate on single volumes, not on 4D data!


In [ ]:
def combine_all_transforms(reference_volume, warp_save_name, is_linear, epi_2_moco=None, epi_2_anat_warp=None, anat_2_standard_warp=None):
    """
    Combines transformations from motion correction to standard space transformation. 
    
    Note: the code operates on a single volume per time 

    Parameters
    ----------
    reference_volume: str
        The reference volume that will define the resolution and field of view for the final image after all transformations are applied.
    warp_save_name: str
        The filename where the combined transformation warp will be saved.
    is_linear: bool
        Indicates whether the final transformation is linear or non-linear.
    epi_2_moco: str
        The motion correction transformation matrix for the EPI volume (located in the .mat folder of the EPI).
    epi_2_anat_warp: str
        The transformation that aligns the EPI volume to the anatomical space, typically including fieldmap correction (non-linear).
    anat_2_standard_warp: str
        The transformation from the anatomical space to standard space (e.g., MNI152). This may be linear or non-linear.

    """
    from fsl.wrappers import convertwarp
    # Import the FSL function 'convertwarp' to combine transformations.

    # Initialize the base arguments for convertwarp using the available transformations.
    args_base = {'premat': epi_2_moco, 'warp1': epi_2_anat_warp}
    # 'premat' is used for the motion correction transformation matrix.
    # 'warp1' is the warp field that aligns the EPI to anatomical space.

    if is_linear:
        # If the transformation to standard space is linear, use 'postmat' for the transformation matrix.
        args_base['postmat'] = anat_2_standard_warp
    else:
        # If the transformation to standard space is non-linear, use 'warp2' for the warp field.
        args_base['warp2'] = anat_2_standard_warp

    # Filter out any None values from the arguments, keeping only the valid transformations.
    args_filtered = {k: v for k, v in args_base.items() if v is not None}

    # Combine the transformations using convertwarp.
    convertwarp(warp_save_name, reference_volume, **args_filtered)
    # The combined transformation is saved as 'warp_save_name', with 'reference_volume' defining the output resolution.

    print("Done with warp conversion")
    # Print a message indicating that the warp conversion is complete.

def apply_transform(reference_volume, target_volume, output_name, transform):
    """
    Applies a warp field to a target volume and resamples it to match the space of the reference volume.

    Parameters
    ----------
    reference_volume: str
        The reference volume used for final interpolation, resampling, and setting the field of view.
    target_volume: str
        The target volume to which the warp will be applied.
    output_name: str
        The filename where the transformed image will be saved.
    transform: str
        The filename of the warp (assumed to be a .nii.gz file).

    See also
    --------
    combine_all_transforms: To see how to build a warp field.
    """
    from fsl.wrappers import applywarp
    # Import the FSL function 'applywarp' to apply the transformation.

    # Apply the warp transformation to the target volume, using the reference volume for resampling.
    applywarp(target_volume, reference_volume, output_name, w=transform, rel=False)
    # 'target_volume': The EPI or image to be transformed.
    # 'reference_volume': Defines the resolution and space for resampling.
    # 'output_name': The file path where the transformed volume will be saved.
    # 'w=transform': Specifies the warp field to apply.
    # 'rel=False': Ensures the warp is applied in absolute coordinates rather than relative.


### Summary
- **combine_all_transforms**: Merges different transformation steps (motion correction, EPI-to-anatomical alignment, and anatomical-to-standard alignment) into a single warp file. This allows you to efficiently apply all transformations in one step.

- **apply_transform**: Uses the combined warp to transform and resample a target image (e.g., EPI data) so it matches the reference space (e.g., the standard MNI space or the subject’s anatomical space).

In combine_all_transforms, setting any transform to None instead of the correct transform will skip the transform step in the total transformation. This way, you should be able to perform quality control. In particular, please ensure that:
- [ ] Applying ONLY motion correction transformation to the first volume yields the expected alignement (so it should be aligned with the \_moco volume.)
- [ ] Applying motion correction + epi -> anat should be aligned to anatomical
- [ ] Finally, applying motion correction + epi > anat + anat > standard should be aligned to the standard
Only once you're convinced these steps are working well should you proceed to standard space. **Remember the 1-10-100 rule! Always perform QC before moving on.**

To help you, we provide you below with the template to do such a thing, so that you don't have to worry too much about the nitty gritty details.
Focus on:
- The reference file to use 
- The transformations to provide (either a file or None)

<b>Notice this is performed only on a single volume. Indeed, if you are debugging you should avoid wasting time applying transformations on entire timeseries to quickly diagnose whether a step is working or failing.</b>

In [ ]:
import time
from fsl.wrappers import applywarp

ref = mni_template
# 'ref' is set to the MNI template, which will be used as the reference space for alignment.

# The first volume of the EPI dataset is selected as the target.
target_epi = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_first-vol')
# This variable points to the path of the first volume extracted from the EPI time series.

split_nbr = '0000'
# A variable to keep track of the volume number being processed. In this case, it's the first volume ('0000').

# We define the path where the combined warp will be saved.
warp_name = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_split' + split_nbr + '_epi_2_std_warp')
# This is where the combined warp (transformation) file for the EPI volume will be saved.

# Define paths to the necessary transformations:

# The EPI to anatomical warp file (with fieldmap corrections included).
func_2_anat = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-sitrep_run-1_bold_anat-space_warp.nii.gz')

# The motion correction matrix specific to the first volume ('0000').
epi_moco = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-sitrep_run-1_bold_moco.mat/', 'MAT_' + split_nbr)

# Timing the combination of transformations
s0 = time.time()

# Combine the motion correction, EPI-to-anatomical, and anatomical-to-standard space transformations
combine_all_transforms(ref, warp_name, True, epi_2_moco=epi_moco, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)
# 'combine_all_transforms' combines the transformations for the volume:
# - The motion correction matrix (epi_moco).
# - The warp from EPI to anatomical space (func_2_anat).
# - The warp from anatomical to standard space (anat_2_mni_trans).
# - The combined warp is saved as 'warp_name'.

s1 = time.time()
# Record the time after combining the transformations.

# Define the output path for the transformed volume
out_vol = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_std_vol' + split_nbr)
# This is where the transformed first volume will be saved after applying the combined warp.

# Apply the transformation using applywarp
applywarp(target_epi, ref, out_vol, w=warp_name, rel=False)
# 'applywarp' applies the combined warp to the target EPI volume ('target_epi'):
# - The reference space is the MNI template ('ref').
# - The output file will be saved as 'out_vol'.
# - 'w=warp_name' specifies the combined warp file.
# - 'rel=False' indicates that absolute coordinates are used for transformation.

s2 = time.time()
# Record the time after applying the transformation.



### Optimizing a bit

We consider <b>grouping up the transformations to apply them</b>. As you will see, computing transformations can take time when they are non linear, whereas applying them is comparatively fast. We will investigate both grouping all transforms together or grouping all transforms which follow motion correction together.


In [ ]:
import time
from fsl.wrappers import applywarp

# Set the reference volume to the MNI template for alignment
ref = mni_template

# Select the target volume: the first volume ('0000') of the EPI dataset
target_epi = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_first-vol')
split_nbr = '0000'  # The volume number we're working with

# Define the name for the warp file that will be saved after combining transformations
warp_name = op.join(preproc_root, 'sub-control01', 'func', 'sub-control1_split' + split_nbr + '_epi_2_std_warp')

# Define paths to the necessary transformations
func_2_anat = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_anat-space_warp.nii.gz')
# Path for the warp from EPI to anatomical space, typically including fieldmap correction

epi_moco = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_moco.mat/', 'MAT_' + split_nbr)
# Path for the motion correction matrix specific to the volume ('0000')

# ----- START OF METHOD 1 -----
# Record the start time for Method 1
s0 = time.time()

# Combine the transformations: motion correction, EPI-to-anatomical, and anatomical-to-standard space
combine_all_transforms(ref, warp_name, True, epi_2_moco=epi_moco, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)

# Record the time after combining the transformations
s1 = time.time()

# Define the output path for the transformed volume
out_vol = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_std_vol' + split_nbr + '_v1')

# Apply the combined warp to the EPI volume using the applywarp command
# subprocess.run(['applywarp', '-i', target_epi, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs'])
subprocess.run(['applywarp', '-i', target_epi, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs'])

# Record the time after applying the transformation
s2 = time.time()

# ----- START OF METHOD 2 -----
# Combine only the transformations post motion correction
combine_all_transforms(ref, warp_name, True, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)

# Record the time after combining the transformations for Method 2
s3 = time.time()

# Define the output path for the transformed volume in Method 2
out_vol = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_std_vol' + split_nbr + '_v2')

# Apply the transformation using the premat option for motion correction matrix
subprocess.run(['applywarp', '-i', target_epi, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs', '--premat={}'.format(epi_moco)])

# Record the time after applying the transformation
s4 = time.time()

# Print the runtime for Method 1: total time and breakdown (combination vs. application)
print('Method 1 runtime:', s2 - s0, '({} for combination, {} to apply)'.format(s1 - s0, s2 - s1))

# Print the runtime for Method 2: total time and breakdown (combination vs. application)
print('Method 2 runtime:', s4 - s2, '({} for combination, {} to apply)'.format(s3 - s2, s4 - s3))


The two produced images are almost identical 

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(ref)
fsleyesDisplay.load(op.join(preproc_root, 'sub-001', 'func', 'sub-control01_task-music_run-1_bold_std_vol' + split_nbr + '_v1'))
fsleyesDisplay.load(op.join(preproc_root, 'sub-001', 'func', 'sub-control01_task-music_run-1_bold_std_vol' + split_nbr + '_v2'))

Why does it matter? Well, just applying a back-of the envelope calculation, the first method takes 122s per volume, while the second method takes 87 seconds to combine **once** the transforms excluding motion correction, and 4 seconds per volume to apply the transforms including motion correction. If we plot the two with an increasing number of volumes, we can see why this quickly becomes relevant:

### Applying the transformation to the entire timeseries at last

With all this in mind, let's now apply our transformation to all our volumes! The steps are:

1. Split our EPI into all individual volumes (remember: applywarp only works on a single 3D image but our EPI is 4D).
2. Combine all transformations from EPI after motion correction all the way to standard space **once**. 
3. Use applywarp for every volume, passing the motion correction transform of this volume and the EPI > standard space warp
4. Combine back all volumes as a single 4D EPI in standard space

Let's make sure you understand why!

In [ ]:
# We will split our starting EPI volume across time 

# original_epi = op.join(bids_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold')
split_target = original_epi
split_name = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_split')

subprocess.run(['fslsplit', split_target, split_name, '-t'])
# This line uses the subprocess module to run the command 'fslsplit', which is an FSL tool for splitting 4D fMRI data.
# 'fslsplit' splits the EPI volume ('split_target') into multiple 3D volumes based on time.
# The resulting files are saved with the prefix defined in 'split_name'. The '-t' flag specifies that the split 
# should occur across the time dimension, meaning each resulting file will correspond to a single time point in the original 4D EPI volume.

In [ ]:
print_dir_tree(bids_root,max_depth=5)

As you can see, a lot of new volumes have appeared. These are the split, individual volumes of our EPI.

Great, let's now combine the different transforms EXCEPT motion correction, with method 2.

In [ ]:
import concurrent.futures
from tqdm import tqdm

# Let's combine the different transforms EXCEPT motion correction!
warp_name = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_epi_moco_2_std_warp')

print("Starting to combine transforms...")
combine_all_transforms(ref, warp_name,  True, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)
print("Done, moving on to application of transforms...")



###########
# Now apply transformation to all our volumes.
# We will remember the volumes as well, to group them back afterwards.
##########

# Notice that we are sorting the volumes here! This is important, to make sure we don't get them in random order :)
split_vols = sorted(glob.glob(op.join(preproc_root, 'sub-control01', 'func', '*_bold_split*')))


# Define a function that wraps subprocess.run()
def run_subprocess(split_vol, vol_nbr):
    """
    SAFETY GOGGLES ON
    This function launches applywarp in parallel to reach complete result quicker

    Parameters
    -----------
    split_vol: str
        Path to the volume on which to apply the transformation
    vol_nbr: str
        Number of the volume in the timeserie. Useful to reorder volumes after the fact, since parallelisation does not honour order.

    Returns
    -------
    out_vol: str
        Path to the transformed volume
    vol_nbr: str
        Number of the volume in the timeserie. Useful to reorder volumes after the fact.
    """
    try:
        split_nbr = split_vol.split('_')[-1].split('.')[0].split('split')[1]
        epi_moco = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_moco.mat/', 'MAT_' + split_nbr)
        out_vol= op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_std_vol' + split_nbr)
        result = subprocess.run(['applywarp', '-i', split_vol, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs', '--premat={}'.format(epi_moco)], check=True)
        return out_vol, vol_nbr
    except subprocess.CalledProcessError as e:
        return f"applywarp for volume '{split_vol}' failed with error: {e.stderr.decode('utf-8')}"


produced_vols = [None]*len(split_vols)
# Initialize ThreadPoolExecutor and the progress bar
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use tqdm to wrap the futures
    with tqdm(total=len(split_vols)) as progress:
        # Launch subprocesses in parallel
        futures = {executor.submit(run_subprocess, vol,i): vol for i,vol in enumerate(split_vols)}

        # Process completed tasks
        for future in concurrent.futures.as_completed(futures):
            out_vol, vol_nbr = future.result()  # Get the result of the subprocess
            produced_vols[vol_nbr] = out_vol
            # Update the progress bar for each completed task
            progress.update(1)

Now we finally have our volumes!

### Grouping the volumes together (Optional)

Our volumes are now individually separated, but we started with a 4D volume.  In theory we should be able to group everything back. The issue is, our fMRI is now interpolated at 1 x 1 x 1 mm3 resolution, so it won't fit in RAM on this virtual machine, where a high performance computing cluster would do this without any issue.

There are two answers to this issue. The first, used by fMRIprep, is to play it smart and actually never modify the fMRI's resolution.

The second is to use to group back the files a memory map, that is to say a file on disk from which we only read the parts we need (VERY useful when you do not have enough RAM). Because writing to disk is very slow, we do it in a batch approach below.

<div class="warning" style='background-color:#C04000; color: #112A46; border-left: solid #C04000 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>⚠️  This part is TIME INTENSIVE and it is OPTIONAL ⚠️ </b></p>
<p style='text-indent: 10px;'>
    This part will take TIME (about an hour, in fact)
</span>
</div>

In [ ]:
import nibabel as nib
import progressbar

first_vol = nib.load(produced_vols[0])
v_shape = first_vol.get_fdata().shape

preproc_root = '/home/jovyan/Data/dataset/ds004226/derivatives/preprocessed_data'
filename = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_std.dat')
large_array = np.memmap(filename, dtype=np.float64, mode='w+', shape=(v_shape[0],v_shape[1],v_shape[2], len(produced_vols)))

batch_size = len(produced_vols)//4

A = np.zeros((v_shape[0],v_shape[1],v_shape[2], batch_size))

with progressbar.ProgressBar(max_value=len(produced_vols)) as bar:
    for batch_i in range(4):
        print('Starting for batch {}/4'.format(batch_i+1))
        start_batch = batch_size * batch_i
        end_batch = min(batch_size * (batch_i+1),len(produced_vols))
        max_len = end_batch - start_batch + 1
        for i in range(start_batch, end_batch):
            vol = nib.load(produced_vols[i])
            A[:,:,:,i-start_batch] = vol.get_fdata()
            bar.update(i)
        large_array[:,:,:, start_batch:end_batch] = A[:,:,:,:max_len]

In [ ]:
# Now, save to Nifti file using Nibabel

# Step 1: Ensure all changes of the memmap are flushed to disk and then close it
#large_array.flush()
#del large_array
print("Done flushing mmap")
large_array = np.memmap(filename, dtype=np.float64, mode='r', shape=(v_shape[0],v_shape[1],v_shape[2], len(produced_vols)))

# Step 2: Modify the header to indicate that we have 4D data, and specify its TR.
header = first_vol.header.copy()  # Copy the header of the first volume (to get right resolution, affine, Q-form etc)
header['dim'][0] = 4  # Specifies that this is a 4D dataset
header['dim'][1:5] = large_array.shape  # Update dimensions (x, y, z, t)
header['pixdim'][4] = 1.5  # Set the TR in the 4th dimension. You can see the TR of the data by looking at your original EPI series with fslhd, remember ;)
print("Done with header")

# Step 3: Create the Nifti1 image and save it to disk
mni_epi = op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_mni.nii.gz')
img = nib.Nifti1Image(large_array, first_vol.affine, first_vol.header)
print("Done creating the image")
img.to_filename(mni_epi)
print("Done writing it to disk")

Finally, we can move on to removing all the temporary files we used, to end up with only one clean Nifti file :)

In [ ]:
os.system('rm -rf {}'.format(op.join(preproc_root, 'sub-control01', 'func', 'sub-control01_task-music_run-1_bold_split*.nii.gz')))

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(ref)
fsleyesDisplay.load(produced_vols[0])

## Where are we?

<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>Field unwarping</td><td style='text-align:justify;'>Correction distortions induced by inhomogeneities of the b0 field through maps acquired specifically to measure this field called fieldmaps.</td><td style='text-align:justify;'>FUGUE (but also FLIRT - see below)</td></tr>
    <tr><td></td><td style='text-align:left;'>Slice-timing correction</td><td style='text-align:justify;'>Accounting for the difference in acquisition between the slices that make up a volume to interpolate back voxels to a fixed time reference.</td><td style='text-align:justify;'>slicetimer</td></tr>
</table>

## Slice correction 

#### Application to real data

We have shown you the basic principle, but the application to real data requires some specific informations.
You need the following ingredients:
- When was each slice acquired in the sequence: **(Slice timing)**
- Along which axis were the slices acquired: **Phase direction**
- How much time we take to acquire all slices: **TR**


In [ ]:
data = get_json_from_file(op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold.json'))
data

This data is actually a dictionary. We can thus extract the slice timing as an array directly from it. For example, to extract TaskName, we would use:
```python
data['TaskName']
```



In [ ]:
slice_timing = data['SliceTiming'] # Replace with the appropriate key (have a look above!)

Now, we might want to know where our slices are, ie along which axis, right? Typically it is along the z-direction, but we're better off if we check! Using FSLeyes, determine how many slices each axis has **for the functional data of interest**. You should thus open the relevant functional file in FSLeyes to answer this question.


<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>Using FSL command line</b></p>
<p style='text-indent: 10px;'>To figure out the dimensions of an MRI image, a faster option - if you have FSL installed directly - is to run the command line command:
    <blockquote>fslhd [your_volume]</blockquote>
This will give you all informations contained within the header of the NIfti file. For example, running the command for our volume will easily allow us to access the slice informations:
    <img src="imgs/fslhd_capture.png"></p>
</span>
</div>
Let's compare now with the amount of slices we have in our acquisition. We can consider simply the number of timings for this

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold.nii.gz'))

In [ ]:
len(slice_timing)

Great, so we know which axis we want, we know the slice timings, but we still need to know the TR. This information is also in the JSON sidecar! Extract it now!

In [ ]:
tr = data['RepetitionTime'] # Extract the TR from the sidecar's appropriate field
tr

To now perform the correction, we need to apply FSL's slicetimer command. For this, we need to save the timings first to their own separate file! Instead of giving the slice timings, we will provide instead the slice **order** (ie which slice was done in which order) and let FSL figure out how to best correct based on this information.

Let's do it.

In [ ]:
slice_order = np.argsort(slice_timing) + 1

# Write to a file the corresponding sorted timings :)
timing_path = op.join(preproc_root,  'sub-001', 'func', 'sub-001_task-sitrep_run-01_slice-timings.txt')
file = open(timing_path, mode='w')
for t in slice_order:
    file.write(str(t) + '\n')
file.close()

Finally we can call slicetimer from a terminal!

In [ ]:
file_to_realign = op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
output_target = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_slice-corr')

subprocess.run(['slicetimer', '-i', file_to_realign, '-o', output_target, '-r', str(tr), '-d', str(3), '--ocustom={}'.format(timing_path)])
#cmd = 'slicetimer -i ' + file_to_realign + ' -o ' + output_target + ' -r ' + str(tr) + ' -d 3 --ocustom=' + timing_path
#os.system(cmd)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(file_to_realign)
fsleyesDisplay.load(output_target)

# MRI and fMRI preprocessing: conclusion

You have reached the end the preprocessing for MRI and fMRI. 

As you can see, there are many fairly involved steps that need to be conducted.

To normalize choices and results, software solutions exist, such as the excellent <a href="https://fmriprep.org/en/stable/">fmriprep</a>, to conduct automatically all steps for you while providing you with quality checks to verify that all went well, as no algorithm can replace your expert eye to inspect potential artefacts and remove them.

These tools are nonetheless precious to help different groups follow same systematic choices of preprocessing, both in parameters and order of application for each method, leading to more reproducible science in the long run. (But they require a big RAM and take hours to run, which is why we've avoided them for the purpose of this tutorial :) )

Let's wrap up what you have learnt.


<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Anatomical</th><td></td><td></td></tr>
    <tr><td></td><td style='text-align:justify;'>Skull stripping</td><td style='text-align:justify;'>Removing skull and surrounding tissues to keep only the brain</td><td style='text-align:justify;'>BET</td></tr>
    <tr><td></td><td style='text-align:justify;'>Segmentation</td><td style='text-align:justify;'>Segmenting brain tissues based on their contrasts</td><td style='text-align:justify;'>FAST</td></tr>
    <tr><td></td><td style='text-align:justify;'>Normalization</td><td style='text-align:justify;'>Mapping participant's brain to a reference brain, making its orientation and scale match so that comparison across participants become feasible.</td><td style='text-align:justify;'>FLIRT</td></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>First few volumes removal</td><td style='text-align:justify;'>Removing volumes for which the B0 field is still not stable and that could contaminate all our data if left unchecked.</td><td style='text-align:justify;'>fslroi</td></tr>
    <tr><td></td><td style='text-align:left;'>Motion correction</td><td style='text-align:justify;'>Realignment of fMRI volumes to a common reference - typically one volume or the average of the volumes - to correct for inter-volume motion. The extracted motion parameters can be used for subsequent analysis (see GLM in one week!)</td><td style='text-align:justify;'>MCFLIRT (which is one suboption of FLIRT in fact)</td></tr>
    <tr><td></td><td style='text-align:left;'>Fieldmap preparation</td><td style='text-align:justify;'>Field maps can be used to create a distortion field to correct...Distortions. </td><td style='text-align:justify;'>topup, FUGUE</td></tr>
    <tr><td></td><td style='text-align:left;'>Coregistration</td><td style='text-align:justify;'>Realignment of fMRI volumes to anatomical space - the subject's own MRI is typically used. We can include susceptibility-distortion correction with fieldmaps. We compute this transformation only for the volume we used as reference in MCFLIRT. Then, we apply it to all other volumes in the EPI.</td><td style='text-align:justify;'>epi_reg (FLIRT with boundary-based registration)</td></tr>
    <tr><td></td><td style='text-align:left;'>Normalization</td><td style='text-align:justify;'>Putting the EPI in a template space, such as MNI. This is done by applying the transformation of anatomical space to MNI space, which was computed in the anatomical preprocessing. Note that we typically like to combine this transform with coregistration to do everything in one go.</td><td style='text-align:justify;'>applywarp to apply combined warps, otherwise (if going transform by transform), FLIRT with applyxfm option</td></tr>
</table>


If you have any more questions, both on these tools and on preprocessing, do not hesitate!